# Ballpark Factor

Load the necessary packages

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

the team names and numbers

In [ ]:
teams =["Bethel", "Goshen", "Grace", "HU", "IWU", "Marian", "MVNU", "SAU", "SFU", "Taylor"]
t_nums = [1629, 1678, 1679, 1688, 1694, 1717, 1736, 1780, 1805, 1784]

Scrape the webpage for each team.

In [41]:
urls = ['http://www.dakstats.com/WebSync/Pages/Team/TeamSchedule.aspx?association=10&sg=MBA&sea=NAIMBA_2019&team=' +
        str(num) for num in t_nums]
#Create a handle, page, to handle the contents of the website
pages = [requests.get(url) for url in urls]
#Store the page as an element tree using BeautifulSoup4
soups = [BeautifulSoup(page.content) for page in pages]

In [42]:
team_tables = [
  [
    [
      [td.get_text(strip=True) for td in tr.find_all('td')] 
      for tr in table.find_all('tr') 
    ]#for each row in each table
    for table in soup.find_all('table') 
  ]#for each table on each webpage
  for soup in soups 
]#for each team's webpage

The for-loop below helps us find which tables on Grace College's webpage.  It appears that the header is the 33rd table, and the actual data is in the 35th table.  It seems like a fair assumption that this will be the same for all of the other teams as well.

In [43]:
for i in range(len(team_tables[2])):
  #print(i, team_tables[2][i])
  #The line ablve is commented out becuause we only needed to run it once to find the location of the data on the webpage.
  pass

0 [['']]
1 [["Browse by SchoolMember LoginFAQ'sScoreboardsWebcastsTournaments", '', '', "Browse by SchoolMember LoginFAQ's", 'Browse by School', 'Member Login', "FAQ's", 'ScoreboardsWebcastsTournaments', 'Scoreboards', 'Webcasts', 'Tournaments', '', '', ''], ['', '', "Browse by SchoolMember LoginFAQ's", 'Browse by School', 'Member Login', "FAQ's"], ['Browse by School', 'Member Login', "FAQ's"], ['ScoreboardsWebcastsTournaments', 'Scoreboards', 'Webcasts', 'Tournaments'], ['Scoreboards', 'Webcasts', 'Tournaments'], [''], ['']]
2 [['', '', "Browse by SchoolMember LoginFAQ's", 'Browse by School', 'Member Login', "FAQ's"], ['Browse by School', 'Member Login', "FAQ's"], ['ScoreboardsWebcastsTournaments', 'Scoreboards', 'Webcasts', 'Tournaments'], ['Scoreboards', 'Webcasts', 'Tournaments'], [''], ['']]
3 [['Browse by School', 'Member Login', "FAQ's"]]
4 [['Scoreboards', 'Webcasts', 'Tournaments']]
5 [['NAIA', 'NAIA', '', 'BSB', 'BSB', '', 'FB', 'FB', '', 'MBB', 'MBB', '', 'MLAX', 'MLAX', '',

In [44]:
headers = [['Date', 'Opponent', 'Location', 'Score', 'Outcome'] for tables in team_tables]
headers[2]

['Date', 'Opponent', 'Location', 'Score', 'Outcome']

In the code below, we use `[:5]` to skip the last column, and we use `[1::2]` because the rows with actual data are the odd rows.

In [45]:
team_rows = [[r[:5] for r in tables[35][1::2]] for tables in team_tables]
team_rows[2][:9]

[['2/27/2019', 'Lourdes (Ohio)', 'N', '3-4', 'L'],
 ['2/27/2019', 'Lourdes (Ohio)', 'N', '4-8', 'L'],
 ['3/2/2019', 'Cornerstone (Mich.)', 'N', '3-4', 'L'],
 ['3/2/2019', 'Trinity Baptist', 'N', '5-1', 'W'],
 ['3/4/2019', 'Michigan-Dearborn', 'N', '13-1', 'W'],
 ['3/5/2019', 'Rochester (Mich.)', 'N', '24-4', 'W'],
 ['3/6/2019', 'Robert Morris (Ill.)', 'N', '10-9', 'W'],
 ['3/8/2019', 'Bethel (Ind.) *', 'N', '13-6', 'W'],
 ['3/9/2019', 'Bethel (Ind.) *', 'N', '14-2', 'W']]

In [46]:
dfc = [pd.DataFrame(columns = headers[i], data = team_rows[i]) for i in range(len(headers))]
dfc[2][:5]

,Date,Opponent,Location,Score,Outcome
0,2/27/2019,Lourdes (Ohio),N,3-4,L
1,2/27/2019,Lourdes (Ohio),N,4-8,L
2,3/2/2019,Cornerstone (Mich.),N,3-4,L
3,3/2/2019,Trinity Baptist,N,5-1,W
4,3/4/2019,Michigan-Dearborn,N,13-1,W


Get only the conference games

In [47]:
conf_df = [df[df.Opponent.str.contains("*", regex = False)] for df in dfc]
conf_df[2][:5]

,Date,Opponent,Location,Score,Outcome
7,3/8/2019,Bethel (Ind.) *,N,13-6,W
8,3/9/2019,Bethel (Ind.) *,N,14-2,W
9,3/9/2019,Bethel (Ind.) *,N,3-1,W
10,3/14/2019,Taylor (Ind.) *,A,5-15,L
11,3/16/2019,Taylor (Ind.) *,A,2-10,L


In the code below, `.copy()` makes a copy of each dataframe, which prevents warning messages.  `.str.replace(r"\(.*\)","")` deletes any parententheses along with anything inside them.  Then `st.split()` is used to split the Scores column into two columns.

After that we use `.assign()` to assign the correct data types.

In [48]:
tidy_conf = conf_df.copy()
for i, df in enumerate(conf_df):
  split_scores = df['Score'].str.replace(r"\(.*\)","").str.split('-', expand = True)
  tidy_conf[i] = df.assign(Score = pd.to_numeric(split_scores[0]),
                           Other_score = pd.to_numeric(split_scores[1]),
                           Opponent = df.Opponent.str.replace(' \*', '', regex= True),
                           Date = pd.to_datetime(df.Date)
                           )
tidy_conf[2][:5]

,Date,Opponent,Location,Score,Outcome,Other_score
7,2019-03-08,Bethel (Ind.),N,13,W,6
8,2019-03-09,Bethel (Ind.),N,14,W,2
9,2019-03-09,Bethel (Ind.),N,3,W,1
10,2019-03-14,Taylor (Ind.),A,5,L,15
11,2019-03-16,Taylor (Ind.),A,2,L,10


In [49]:
conf_h = [df[df.Location.str.contains("H", regex = False)] for df in tidy_conf]
conf_a = [df[df.Location.str.contains("A", regex = False)] for df in tidy_conf]

In [50]:
h_runs_per_game = [(df.Score.sum() + df.Other_score.sum())/len(df.index) for df in conf_h]
a_runs_per_game = [(df.Score.sum() + df.Other_score.sum())/len(df.index) for df in conf_a]
park_factor = [h_runs_per_game[i]/a_runs_per_game[i] for i in range(len(headers))]
park_factor

[0.963768115942029,
 1.1224268689057422,
 0.8870967741935484,
 0.7320261437908496,
 1.0357675111773472,
 0.7938931297709924,
 0.8571428571428571,
 1.15,
 1.3483365949119375,
 1.3363844393592679]